In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.api as sms
import numpy as np
from math import ceil
from scipy.stats import shapiro, levene, mannwhitneyu

df=pd.read_csv(fr"/Users/choyy/Documents/AB_testing/ab_2/ab_data.csv", dtype={'user_id':str})
df

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
...,...,...,...,...,...
294475,734608,45:03.4,control,old_page,0
294476,697314,20:29.0,control,old_page,0
294477,715931,40:24.5,treatment,new_page,0
294478,759899,20:29.0,treatment,new_page,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  object
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 11.2+ MB


In [3]:
control = df.query("group == 'control' & landing_page == 'new_page'")
control_id = control['user_id'].tolist()
treatment = df.query("group == 'treatment' and landing_page == 'old_page'")
treatment_id = treatment['user_id'].tolist()

# cannot like this, as there is duplicate for the user_id
# df=df[~df['user_id'].isin(control_id)]
# df=df[~df['user_id'].isin(treatment_id)]
# df.shape

# need to use filter

df=df[(df['group'] == 'treatment') & (df['landing_page'] == 'new_page') |
              (df['group'] == 'control') & (df['landing_page'] == 'old_page')]
df.shape

(290587, 5)

In [4]:
df

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
...,...,...,...,...,...
294475,734608,45:03.4,control,old_page,0
294476,697314,20:29.0,control,old_page,0
294477,715931,40:24.5,treatment,new_page,0
294478,759899,20:29.0,treatment,new_page,0


In [5]:
duplicate=df.groupby('user_id').agg(count_id=('user_id','count')).reset_index()

df=df.merge(duplicate, on='user_id',how='left')

df = df[~(df['count_id'] > 1)]
# drop userid approve more than one
df = df.drop_duplicates(subset= 'user_id', keep= False)



In [6]:
grouped=df.groupby(['group','landing_page']).agg(count_group=('user_id','count')).reset_index()
grouped

,group,landing_page,count_group
0,control,old_page,145274
1,treatment,new_page,145309


In [7]:
mean_df=df.groupby(['group','landing_page']).agg({'converted': 'mean'}).reset_index()
mean_df

,group,landing_page,converted
0,control,old_page,0.120386
1,treatment,new_page,0.118809


In [8]:
# check whether it is normally distributed

test_stat, pvalue = shapiro(df.loc[df["landing_page"] == "old_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.3796349763870239


C:\Users\choyy\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [9]:
test_stat, pvalue = shapiro(df.loc[df["landing_page"] == "new_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.37687963247299194


p-value < 0.05, so assumption of normality is not provided. we will use non-parametric test(mannwhitneyu test)

In [10]:
#Variance Homogeneity

test_stat, pvalue = levene(df.loc[df["landing_page"] == "new_page", "converted"],
                           df.loc[df["landing_page"] == "old_page", "converted"])
print("p-value:",pvalue)  
print("test_stat:",test_stat)


p-value: 0.190115619416887
test_stat: 1.7167294322514532


 p-value not reject Ho, Variances are homogeneous

In [11]:
#  Hypothesis Testing

test_stat, pvalue = mannwhitneyu(df.loc[df["landing_page"] == "new_page", "converted"],
                                 df.loc[df["landing_page"] == "old_page", "converted"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat = 10538160450.5000, p-value = 0.1901


p-value(0.2323) >0.05
Not reject null hypothesis, no difference between new and old page